In [2]:
import time
import adodbapi
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
start_time = time.time()

In [4]:
client_manuf = ["Bel"]
client_brands = ["La Vache Qui Rit","Kiri","Boursin"]
 
decimals = 2
sign = "After"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
 
prodORitem = "Item"
categories = ["Total Fromage"]
sectors = ["Soft Cheese", "Aperitif", "Ingredient A Chaud"]
segments = ["Enfant", "Frais A Tartiner", "Salade"]
subsegments= []
subcategories= []
 
 
national = False
customareas= ""
areas = ["RETAILER"]

regions_RET  = ["Carrefour", "Intermarche"]
channels_RET = []
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []


data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending December 2024"


ManufOrTopC ="Top Companies"
BrandOrTopB="Top Brands"

end_date = "2025-01-01"



National=[]
regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST
brands_only = True  # Get the Data of SKU Share by brands level only

entity_hierarchy = [
    ("Area",National),
    ("Region", regions),
    ("Channel", channels),
    ("Market", markets)
]
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
]

direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories))
                ,"Segment":list(zip(["Sector"]*len(sectors),sectors))
                #,"SubSegment":list(zip(["Segment"]*len(sectors),sectors))
                #,"SubCategory":list(zip(["Segment"]*len(segments),segments))
                }

server = "powerbi://api.powerbi.com/v1.0/myorg/Groupe Bel"
dataset_name = "Bel France Dataset"
conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};"

print(conn_str)
path=os.path.join(os.getcwd(),"Financilas Datasets NewEX")

past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
past_3_months = pd.date_range(end=end_date, periods=3, freq='ME').strftime('%b-%y').tolist()
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"
p3m_dax = "{" + ", ".join(f'"{date}"' for date in past_3_months) + "}"


Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Groupe Bel;Initial Catalog=Bel France Dataset;


### Slide 67(brands_client_dfs)

In [4]:

def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type):
    outputdic = {}
    columns=["Gross Margin %","Gross Margin % IYA","Gross Profit","Net Sales","Net Sales/Kg","Net Sales/Kg IYA","Volume Sales","Trade Margin %","Trade Profit","Trade Profit/Kg","Av Price/KG","Value Sales IYA","Value Sales","Total WoB %"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{BrandorManuf}],  
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",
        Products[{ManufOrTopC}] = "{client_manuf[0]}",
        
        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {p12m_dax},
            Calendar[MonthYear]
        ),
        TREATAS(
            {{"{entity_name}"}},
            Market[{area}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "{hierby}"   
        )
    )   
    """
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{hierby}]),  -- dummy base just to return a single row
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{ManufOrTopC}] = "{client_manuf[0]}",
            
            Products[{hierby}] = "{entity_type}",
            TREATAS(
                {p12m_dax},
                Calendar[MonthYear] 
            ),
            TREATAS(
                {{"{entity_name}"}},
                Market[{area}]
            ),
            FILTER(
                'Scope', 
                'Scope'[Scope] = "{hierby}"   
            )
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
                
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()        
                    
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
            dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
            dt[df.columns[0]] = 'Grand Total'            
            df = pd.concat([df, dt], ignore_index=True)
            df = df.iloc[:, :-1]

            
            outputdic[f"{client_manuf[0]} | {entity_type} | {entity_name}"] = df
            print(f"Query executed successfully for {entity_type} {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")
    return outputdic

# List of all entities to process

brands_client_dfs = {}
# **Threaded Execution**
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for area, entity_list in entity_hierarchy:
        for entity in entity_list:
            for hierby, hier_values in hierarchy_levels:
                    for value in hier_values:
                        futures.append(executor.submit(execute_dax_query, f"{BrandOrTopB}",entity, area, hierby, value))
               
    # Wait for all tasks to complete
    for future in futures:
        result=future.result()
        brands_client_dfs.update(result)

pd.to_pickle(brands_client_dfs, os.path.join(path,"brands_client_dfs.pkl"))


Query executed successfully for Frais A Tartiner Carrefour.
Query executed successfully for Total Fromage Intermarche.
Query executed successfully for Salade Carrefour.
Query executed successfully for Enfant Carrefour.
Query executed successfully for Soft Cheese Carrefour.
Query executed successfully for Ingredient A Chaud Carrefour.
Query executed successfully for Aperitif Carrefour.
Query executed successfully for Total Fromage Carrefour.
Query executed successfully for Aperitif Intermarche.
Query executed successfully for Soft Cheese Intermarche.
Query executed successfully for Ingredient A Chaud Intermarche.
Query executed successfully for Salade Intermarche.
Query executed successfully for Frais A Tartiner Intermarche.
Query executed successfully for Enfant Intermarche.


### Slide 68 & 69 & 71 & 72

In [5]:

def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type,Monthyear,ke):
    outputdic = {}

    columns=["Value Sales","Value Sales IYA","Trade Margin %","Trade Margin % DYA","Net Sales","Net Sales/Kg","Gross Profit/Kg","Gross Margin %","Trade Profit IYA","VSOD IYA","Av Price/KG","VAT/Kg","Total COGS/Kg","Trade Profit/Kg","Rate of Sales","WD","Total WoB %"]

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{ke}], 
            {column_exprs}            
        ),
        Products[Category] = "{categories[0]}",
        Products[{ManufOrTopC}] = "{client_manuf[0]}",

        TREATAS(
            {Monthyear},
            Calendar[MonthYear]
        ),
        TREATAS(
            {{"{entity_name}"}},
            Market[{area}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "Category"   
        )
    )   
    """
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),  -- dummy base just to return a single row
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{ManufOrTopC}] = "{client_manuf[0]}",
            TREATAS(
                {Monthyear},
                Calendar[MonthYear]
            ),
            TREATAS(
                {{"{entity_name}"}},
                Market[{area}]
            ),
            FILTER(
                'Scope', 
                'Scope'[Scope] = "Category"   
            )
        )   
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
                
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()        
                    
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
            dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
            dt[df.columns[0]] = 'Grand Total'            
            df = pd.concat([df, dt], ignore_index=True)
            df = df.iloc[:, :-1]

            
            outputdic[f"{categories[0]} | {client_manuf[0]} | {entity_name}"] = df
            print(f"Query executed successfully for {entity_type} {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")
    return outputdic

# List of all entities to process

brands_client_dfs = {}
def process_dax_queries(entity_hierarchy, hierarchy_levels,Monthyear):
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for ke, val in direct_parent.items():
            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    for area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key =  f"{client_manuf[0]} | {value} | {entity}"
                            ordered_keys.append(key) 
                            future = executor.submit(execute_dax_query, f"{BrandOrTopB}", entity, area, hierby, value,Monthyear,ke)
                            futures[future] = key


            temp_results = {}
            for future in as_completed(futures):
                key = futures[future]
                result = future.result()
                dfs_results.update(result)


            # Insert results in original order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            if  Monthyear== p12m_dax:
                filename =  f"{ke}_P12M_client_dfs.pkl"
            else:
                filename =  f"{ke}_P3M_client_dfs.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy, hierarchy_levels,p12m_dax)
process_dax_queries(entity_hierarchy, hierarchy_levels,p3m_dax)



Query executed successfully for Soft Cheese Intermarche.
Query executed successfully for Ingredient A Chaud Intermarche.
Query executed successfully for Aperitif Intermarche.
Query executed successfully for Total Fromage Intermarche.
Query executed successfully for Total Fromage Carrefour.
Query executed successfully for Aperitif Carrefour.
Query executed successfully for Ingredient A Chaud Carrefour.
Query executed successfully for Soft Cheese Carrefour.
Query executed successfully for Enfant Carrefour.
Query executed successfully for Frais A Tartiner Carrefour.
Query executed successfully for Salade Carrefour.
Query executed successfully for Salade Intermarche.
Query executed successfully for Frais A Tartiner Intermarche.
Query executed successfully for Enfant Intermarche.
All DataFrames saved to c:\Users\aleaa\Desktop\Slide-Automate Extraction\Financials\Financilas Datasets NewEX\Sector_P12M_client_dfs.pkl.
Query executed successfully for Ingredient A Chaud Intermarche.
Query execut

### Slide 70 &73 &74(Product P12M &P3M)

In [6]:

def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type,Monthyear):
    outputdic = {}
    columns=["WD","Gross Profit/Unit","VAT/Unit","Net Sales","Net Sales/Kg","Net Sales/Unit","Gross Margin %","Trade Margin %","Trade Profit/Unit","Av Price/Unit","Total COGS/Unit",f"{prodORitem} Sales Rate","Value Sales IYA"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{BrandorManuf}], 
            Products[{prodORitem}], 
            {column_exprs}            
        ),
        Products[Category] = "{categories[0]}",
        Products[{ManufOrTopC}] = "{client_manuf[0]}",

        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {Monthyear},
            Calendar[MonthYear]
        ),
        TREATAS(
            {{"{entity_name}"}},
            Market[{area}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "{hierby}"   
        )
    )   
    """
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{hierby}]),  -- dummy base just to return a single row
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{ManufOrTopC}] = "{client_manuf[0]}",
            Products[{hierby}] = "{entity_type}",
            TREATAS(
                {Monthyear},
                Calendar[MonthYear]
            ),
            TREATAS(
                {{"{entity_name}"}},
                Market[{area}]
            ),
            FILTER(
                'Scope', 
                'Scope'[Scope] = "{hierby}"   
            )
        )   
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
                
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()        
                    
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
            dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
            dt[df.columns[0]] = 'Grand Total'            
            df = pd.concat([df, dt], ignore_index=True)
            df = df.iloc[:, :-1]

            
            outputdic[f"{client_manuf[0]} | {entity_type} | {entity_name}"] = df
            print(f"Query executed successfully for {entity_type} {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")
    return outputdic

# List of all entities to process

brands_client_dfs = {}
def process_dax_queries(entity_hierarchy, hierarchy_levels,Monthyear):
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            for value in hier_values:
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        key =  f"{client_manuf[0]} | {value} | {entity}"
                        ordered_keys.append(key)  # <-- Add this line
                        future = executor.submit(execute_dax_query, f"{BrandOrTopB}", entity, area, hierby, value,Monthyear)
                        futures[future] = key


        temp_results = {}
        for future in as_completed(futures):
            key = futures[future]
            result = future.result()
            dfs_results.update(result)


        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        if  Monthyear== p12m_dax:
            filename =  f"product_P12M_dfs.pkl"
        else:
            filename =  f"product_P3M_dfs.pkl"

        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy, hierarchy_levels,p12m_dax)
process_dax_queries(entity_hierarchy, hierarchy_levels,p3m_dax)



Query executed successfully for Ingredient A Chaud Intermarche.
Query executed successfully for Aperitif Intermarche.
Query executed successfully for Total Fromage Intermarche.
Query executed successfully for Aperitif Carrefour.
Query executed successfully for Ingredient A Chaud Carrefour.
Query executed successfully for Total Fromage Carrefour.
Query executed successfully for Soft Cheese Intermarche.
Query executed successfully for Soft Cheese Carrefour.
Query executed successfully for Enfant Intermarche.
Query executed successfully for Frais A Tartiner Carrefour.
Query executed successfully for Frais A Tartiner Intermarche.
Query executed successfully for Enfant Carrefour.
Query executed successfully for Salade Carrefour.
Query executed successfully for Salade Intermarche.
All DataFrames saved to c:\Users\aleaa\Desktop\Slide-Automate Extraction\Financials\Financilas Datasets NewEX\product_P12M_dfs.pkl.
Query executed successfully for Total Fromage Intermarche.
Query executed successf

### DataFrames_By Markets

In [7]:
def execute_dax_query(hierby, Monthyear, area, entity_type=None, row_field=None):
    outputdic = {}
    key = f"{client_manuf[0]} | {entity_type}"

    columns = [
        "Gross Margin %", "Gross Margin % IYA", 'Net Sales',
        "Net Sales/Kg", "Net Sales/Kg IYA", "Value Sales IYA"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Market, {row_field}),
                {column_exprs}
            ),
            
            TREATAS({Monthyear}, Calendar[MonthYear]),
            FILTER(Products, Products[Category] = "{categories[0]}"),
            FILTER(Products, Products[{ManufOrTopC}] = "{client_manuf[0]}"),
            FILTER(Products, Products[{hierby}] = "{entity_type}"),
            Filter(Market,Market[Area]="{area}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{ManufOrTopC}] = "{client_manuf[0]}",
            Products[{hierby}] = "{entity_type}",
            TREATAS({Monthyear}, Calendar[MonthYear]),
            Filter(Market,Market[Area]="{area}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            result_columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=result_columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        if grandtotal_data:
            dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
            dt[df.columns[0]] = 'Grand Total'
            df = pd.concat([df, dt], ignore_index=True)
            df = df.iloc[:, :-1]  # Remove duplicate column if needed

        outputdic[key] = df
        print(f"Query executed successfully for {hierby} | {row_field}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(hierarchy_levels, Monthyear, area, row_list):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for row_field in row_list:
            dfs_results = {}
            futures = [
                executor.submit(execute_dax_query, hierby, Monthyear, area, value, row_field)
                for hierby, hier_values in hierarchy_levels
                for value in hier_values
            ]

            for future in as_completed(futures):
                try:
                    result = future.result()
                    dfs_results.update(result)
                except Exception as e:
                    print(f"Error processing query: {e}")

            if area == "RETAILER":
                if Monthyear==p12m_dax:
                    filename = "retailers_P12M_dfs.pkl"
                else:
                    filename = "retailers_P3M_dfs.pkl"
            elif area == "CHANNEL":
                if Monthyear==p12m_dax:
                    filename = "channel_P12M_dfs.pkl"
                else:    
                    filename = "channel_P3M_dfs.pkl"
                    
            elif area == customareas:
                if Monthyear==p12m_dax:
                    filename = "cust_P12M_dfs.pkl"
                else:    
                    filename = "cust_P3M_dfs.pkl"
                    
            else:
                filename = f"{area.lower()}_dfs.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"Saved DataFrames for {row_field} to {output_file}.")


# Final Calls
if "RETAILER" in areas:
    RET_list = []
    if regions_RET: RET_list.append('Market[Region]')
    if channels_RET: RET_list.append('Market[Channel]')
    if market_RET: RET_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, p12m_dax, "RETAILER", RET_list)
    process_dax_queries(hierarchy_levels, p3m_dax, "RETAILER", RET_list)
    

if "CHANNEL" in areas:
    CHA_list = []
    if regions_CHAN: CHA_list.append('Market[Region]')
    if channels_CHAN: CHA_list.append('Market[Channel]')
    if market_CHAN: CHA_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, p12m_dax, "CHANNEL", CHA_list)
    process_dax_queries(hierarchy_levels, p3m_dax, "CHANNEL", CHA_list)
    

if customareas in areas:
    CUST_list = []
    if regions_CUST: CUST_list.append('Market[Region]')
    if channels_CUST: CUST_list.append('Market[Channel]')
    if market_CUST: CUST_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, p12m_dax, customareas, CUST_list)
    process_dax_queries(hierarchy_levels, p3m_dax, customareas, CUST_list)
    


Query executed successfully for Segment | Market[Region].
Query executed successfully for Sector | Market[Region].
Query executed successfully for Sector | Market[Region].
Query executed successfully for Segment | Market[Region].
Query executed successfully for Segment | Market[Region].
Query executed successfully for Category | Market[Region].
Query executed successfully for Sector | Market[Region].
Saved DataFrames for Market[Region] to c:\Users\aleaa\Desktop\Slide-Automate Extraction\Financials\Financilas Datasets NewEX\retailers_P12M_dfs.pkl.
Query executed successfully for Sector | Market[Region].
Query executed successfully for Segment | Market[Region].
Query executed successfully for Segment | Market[Region].
Query executed successfully for Sector | Market[Region].
Query executed successfully for Sector | Market[Region].
Query executed successfully for Category | Market[Region].
Query executed successfully for Segment | Market[Region].
Saved DataFrames for Market[Region] to c:\U

### Slide 75(Manuf_dfs)

In [ ]:
def execute_dax_query(entity_name, area, hierby, entity_type, ke):
    outputdic = {}
    
    columns = [
        "Volume Sales", "Value Share", "Value Sales", "Value Sales IYA", "Trade Margin %",
        "Net Sales", "Net Sales/Kg", "Gross Profit/Kg", "Gross Margin %", "Gross to Net IYA",
        "VSOD IYA", "Av Price/KG", "VAT/Kg", "Total COGS/Kg", "Trade Profit/Kg",
        "Rate of Sales", "WD", "Total WoB %", "Trade Profit", "Gross Profit"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    base_filters = f"""
        Products[Category] = "{categories[0]}",
        TREATAS({p12m_dax}, Calendar[MonthYear]),
        TREATAS({{ "{entity_name}" }}, Market[{area}]),
        FILTER('Scope', 'Scope'[Scope] = "Category")
    """

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{ManufOrTopC}], 
            Products[{ke}], 
            {column_exprs}
        ),
        {base_filters}
    )
    """

    totalcol_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{ManufOrTopC}]), 
                {column_exprs}
            ),
            {base_filters}
        )
        """


    grandtotal_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            VALUES(Products[Category]),
            {column_exprs}
        ),
        {base_filters}
    )
    """

    try:
        # Main Data
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_out = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns_out)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        # Total by entity
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(totalcol_query)
            totalcol_columns = [desc[0] for desc in cursor.description]
            totalcol_data = cursor.fetchall()
        maintotal_df = pd.DataFrame(totalcol_data, columns=totalcol_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]

        maintotal_df["Top Companies"] = maintotal_df["Top Companies"].astype(str) + " Total"

        # Grand total
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
        dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
        dt[df.columns[0]] = 'Grand Total'

        # Combine all
        df_with_totals = pd.concat([maintotal_df, dt], ignore_index=True)
        
        df= pd.concat([df, df_with_totals], ignore_index=True)
        df = df.iloc[:, :-1]

        outputdic[f"{categories[0]} | {entity_name}"] = df
        
        print(f"Query executed successfully for {entity_type} {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for ke, val in direct_parent.items():
            futures = {}
            ordered_keys = []
            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    for area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{categories[0]} | {entity}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query, entity, area, hierby, value, ke)
                            futures[future] = key

            temp_results = {}
            for future in as_completed(futures):
                key = futures[future]
                try:
                    result = future.result()
                    temp_results.update(result)
                except Exception as e:
                    print(f"Failed to process {key}: {e}")

            dfs_results = {}
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            filename = f"manuf_{ke}_dfs"
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"All DataFrames saved to {output_file}.")
process_dax_queries(entity_hierarchy, hierarchy_levels)


Database error for Total Fromage Intermarche: (-2147352567, 'Exception occurred.', (0, 'Microsoft OLE DB Provider for Analysis Services.', 'Query (5, 33) Too many arguments were passed to the VALUES function. The maximum argument count for the function is 1. \n \nTechnical Details: \nRootActivityId: d7f0e17e-69e7-4f51-9e92-65a73ad6521c \nDate (UTC): 5/11/2025 3:08:14 PM', None, 0, -2147467259), None)
Command:

        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                Values(Products,Products[Products[Sector]), 
                "Volume Sales", COALESCE([Volume Sales], 0), "Value Share", COALESCE([Value Share], 0), "Value Sales", COALESCE([Value Sales], 0), "Value Sales IYA", COALESCE([Value Sales IYA], 0), "Trade Margin %", COALESCE([Trade Margin %], 0), "Net Sales", COALESCE([Net Sales], 0), "Net Sales/Kg", COALESCE([Net Sales/Kg], 0), "Gross Profit/Kg", COALESCE([Gross Profit/Kg], 0), "Gross Margin %", COALESCE([Gross Margin %], 0), "Gross to Net IYA", COALESCE(

In [9]:
loaded_data = {}
datasets_path =os.getcwd()+"\\Financilas Datasets NewEX\\"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

            Top Companies  Volume Sales  Value Share   Value Sales  \
0        All Others Total       3516484     0.039107  5.212133e+07   
1   Triballat Rians Total        132740     0.002362  3.148129e+06   
2          Savencia Total      14305942     0.139888  1.864428e+08   
3          Lactalis Total      18687413     0.167246  2.229049e+08   
4               Bel Total       7417193     0.078095  1.040846e+08   
5             Laita Total        882968     0.007774  1.036150e+07   
6          Mondelez Total        360225     0.003140  4.184546e+06   
7         Entremont Total       4157305     0.047500  6.330757e+07   
8      Casa Azzurra Total        828450     0.009934  1.323947e+07   
9            Eurial Total       3321942     0.032780  4.368945e+07   
10         Milleret Total        874263     0.007428  9.900266e+06   
11  Triballat Noyal Total        145668     0.001649  2.197350e+06   
12              Mdd Total      63087830     0.455712  6.073713e+08   
13          Upfield 